# M4 FAULT 10

IMPORTAZIONE LIBRERIE

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import scipy.signal as signal
from scipy.fft import fft, fftfreq
from scipy.signal import find_peaks

CAMBIO PATH PER ANALISI RISPETTO AL LOG DI VOLO CONSIDERATO

In [2]:
path_file = os.path.abspath(os.getcwd())
print(path_file)
os.chdir(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\M4\FAULT_M4_10


SETTAGGIO DELLA VARIABILE RELATIVA AL GUASTO

In [3]:
guasto=2

## RCOU
VARIABILE RCOU POTENZA MOTORI.

Andiamo a prendere una potenza media di 1350 per considerare il momento il cui inizia/finisce il volo

In [4]:
rcou_m4_fault10 = pd.read_csv("RCOU.csv")
rcou_m4_fault10 = rcou_m4_fault10.astype("float64")
rcou_m4_fault10 = rcou_m4_fault10.drop(['LineNo','C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8'], axis=1)

potenza = 1350

rcou_m4_fault10 = rcou_m4_fault10[((rcou_m4_fault10['C9'] >= potenza) & (rcou_m4_fault10['C10'] >= potenza) & (rcou_m4_fault10['C11'] >= potenza) & (rcou_m4_fault10['C12'] >= potenza) & (rcou_m4_fault10['C13'] >= potenza) & (rcou_m4_fault10['C14']>= potenza))]
rcou_m4_fault10=rcou_m4_fault10.reset_index(drop=True)
display(rcou_m4_fault10)

,TimeUS,C9,C10,C11,C12,C13,C14
0,71672974.0,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0
1,71717439.0,1535.0,1415.0,1486.0,1471.0,1540.0,1410.0
2,71727490.0,1354.0,1591.0,1516.0,1453.0,1389.0,1567.0
3,71738394.0,1353.0,1604.0,1506.0,1478.0,1368.0,1595.0
4,71751266.0,1407.0,1583.0,1550.0,1448.0,1461.0,1539.0
...,...,...,...,...,...,...,...
5611,140815408.0,1599.0,1404.0,1593.0,1412.0,1559.0,1455.0
5612,140825207.0,1593.0,1406.0,1594.0,1404.0,1558.0,1450.0
5613,140836705.0,1581.0,1415.0,1590.0,1403.0,1543.0,1461.0
5614,140848285.0,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0


Andiamo a prendere i tempi di inizio e di fine, così da poter effettuare il trimming delle altre tabelle

In [5]:
min=rcou_m4_fault10['TimeUS'][0]
max=rcou_m4_fault10['TimeUS'][len(rcou_m4_fault10)-1]
print(max)
print(min)

140859141.0
71672974.0


## ATT & XKF
### XKF
XKF utilizzato per fare poi il confronto con i valori di roll, pitch e yaw presenti att

In [6]:
xkf1_m4_fault10 = pd.read_csv("XKF1_0.csv")
xkf1_m4_fault10 = xkf1_m4_fault10.astype("float64")
xkf1_m4_fault10 = xkf1_m4_fault10.drop(['LineNo','C','VN','VE','VD','dPD','PN','PE','PD','GX','GY','GZ','OH'], axis=1)
xkf1_m4_fault10 = xkf1_m4_fault10[((xkf1_m4_fault10['TimeUS'] >= min) & (xkf1_m4_fault10['TimeUS'] <= max	))]
xkf1_m4_fault10 = xkf1_m4_fault10.reset_index(drop=True)
print(xkf1_m4_fault10)

           TimeUS  Roll  Pitch     Yaw
0      71683864.0 -0.46  -0.40  355.35
1      71694656.0 -0.10  -0.45  355.36
2      71704946.0  0.10  -0.47  355.35
3      71715377.0  0.05  -0.48  355.34
4      71725387.0 -0.14  -0.40  355.35
...           ...   ...    ...     ...
6282  140813337.0  0.45   4.06    0.42
6283  140823342.0  0.49   3.88    0.36
6284  140834825.0  0.52   3.74    0.39
6285  140846172.0  0.56   3.73    0.45
6286  140856511.0  0.63   3.68    0.55

[6287 rows x 4 columns]


### ATT
Andiamo a prendere dal csv relativo ad ATT i dati relativi alle variabili di roll, pitch e yaw

In [7]:
att_m4_fault10 = pd.read_csv("ATT.csv")
att_m4_fault10 = att_m4_fault10.astype("float64")
att_m4_fault10 = att_m4_fault10.drop(['LineNo','ErrRP', 'ErrYaw', 'AEKF'], axis=1)
att_m4_fault10 = att_m4_fault10[((att_m4_fault10['TimeUS'] >= min) & (att_m4_fault10['TimeUS'] <= max	))]
att_m4_fault10=att_m4_fault10.reset_index(drop=True)
print(att_m4_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw     Yaw
0      71685046.0     0.07 -0.46      0.18  -0.40  355.26  355.36
1      71695745.0     0.04 -0.10      0.17  -0.45  355.26  355.37
2      71706645.0     0.03  0.10      0.16  -0.47  355.26  355.36
3      71716773.0     0.02  0.05      0.15  -0.48  355.26  355.35
4      71726805.0     0.00 -0.14      0.14  -0.40  355.26  355.36
...           ...      ...   ...       ...    ...     ...     ...
6282  140814739.0     1.13  0.45      3.87   4.06    1.35    0.42
6283  140824626.0     1.13  0.49      3.77   3.88    1.35    0.36
6284  140835923.0     1.12  0.52      3.70   3.74    1.35    0.39
6285  140847431.0     1.11  0.56      3.62   3.73    1.34    0.45
6286  140858295.0     1.09  0.63      3.54   3.68    1.34    0.55

[6287 rows x 7 columns]


Tramite un ciclo for si va a vedere se i valori di XKF corrispondono con quelli di ATT in caso contrario fa una media tra i due

In [8]:
from numpy import empty

if len(att_m4_fault10)>len(xkf1_m4_fault10):
    to_add=att_m4_fault10[len(xkf1_m4_fault10):]
    att_m4_fault10=att_m4_fault10[:len(xkf1_m4_fault10)]

if len(xkf1_m4_fault10)>len(att_m4_fault10):
    to_add=xkf1_m4_fault10[len(att_m4_fault10):]

for idx,i in enumerate(att_m4_fault10['Roll']):
    if(xkf1_m4_fault10['Roll'][idx] != i):
        att_m4_fault10['Roll'][idx] = ((att_m4_fault10['Roll'][idx] + xkf1_m4_fault10['Roll'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Pitch']):
    if(xkf1_m4_fault10['Pitch'][idx] != i):
        att_m4_fault10['Pitch'][idx] = ((att_m4_fault10['Pitch'][idx] + xkf1_m4_fault10['Pitch'][idx])/2)

for idx,i in enumerate(att_m4_fault10['Yaw']):
    if(xkf1_m4_fault10['Yaw'][idx] != i):
        att_m4_fault10['Yaw'][idx] = ((att_m4_fault10['Yaw'][idx] + xkf1_m4_fault10['Yaw'][idx])/2)
if 'to_add' in globals():
    att_m4_fault10=pd.concat([att_m4_fault10,to_add])

print(att_m4_fault10)

           TimeUS  DesRoll  Roll  DesPitch  Pitch  DesYaw      Yaw
0      71685046.0     0.07 -0.46      0.18  -0.40  355.26  355.355
1      71695745.0     0.04 -0.10      0.17  -0.45  355.26  355.365
2      71706645.0     0.03  0.10      0.16  -0.47  355.26  355.355
3      71716773.0     0.02  0.05      0.15  -0.48  355.26  355.345
4      71726805.0     0.00 -0.14      0.14  -0.40  355.26  355.355
...           ...      ...   ...       ...    ...     ...      ...
6282  140814739.0     1.13  0.45      3.87   4.06    1.35    0.420
6283  140824626.0     1.13  0.49      3.77   3.88    1.35    0.360
6284  140835923.0     1.12  0.52      3.70   3.74    1.35    0.390
6285  140847431.0     1.11  0.56      3.62   3.73    1.34    0.450
6286  140858295.0     1.09  0.63      3.54   3.68    1.34    0.550

[6287 rows x 7 columns]


## IMU
VALORI DI ACCELERAZIONE E DEL GIROSCOPIO

Per le 3 tabelle di IMU si sono presi i soli valori relativi a GyrX, GyrY, GyrZ, AccX, AccY, AccZ.
In seguito si è poi creata una singola tabella popolandola con la media dei valori delle tre tabelle.

In [10]:
imu_0_m4_fault10 = pd.read_csv("IMU_0.csv")
imu_0_m4_fault10 = imu_0_m4_fault10.astype("float64")
imu_0_m4_fault10 = imu_0_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_0_m4_fault10 = imu_0_m4_fault10[((imu_0_m4_fault10['TimeUS'] >= min) & (imu_0_m4_fault10['TimeUS'] <= max))]

imu_1_m4_fault10 = pd.read_csv("IMU_1.csv")
imu_1_m4_fault10 = imu_1_m4_fault10.astype("float64")
imu_1_m4_fault10 = imu_1_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_1_m4_fault10 = imu_1_m4_fault10[((imu_1_m4_fault10['TimeUS'] >= min) & (imu_1_m4_fault10['TimeUS'] <= max))]

imu_2_m4_fault10 = pd.read_csv("IMU_2.csv")
imu_2_m4_fault10 = imu_2_m4_fault10.astype("float64")
imu_2_m4_fault10 = imu_2_m4_fault10.drop(['LineNo','I', 'EG', 'EA', 'T', 'GH', 'AH', 'GHz', 'AHz'], axis=1)
imu_2_m4_fault10 = imu_2_m4_fault10[((imu_2_m4_fault10['TimeUS'] >= min) & (imu_2_m4_fault10['TimeUS'] <= max))]

imu_m4_fault10 = pd.concat((imu_0_m4_fault10, imu_1_m4_fault10, imu_2_m4_fault10))
imu_m4_fault10=imu_m4_fault10.groupby(imu_m4_fault10.TimeUS, as_index=False).mean()

display(imu_m4_fault10)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ
0,71675210.0,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380
1,71677358.0,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750
2,71679642.0,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288
3,71684827.0,-0.014572,0.018070,0.016195,-0.339925,-0.366561,-9.819399
4,71690467.0,-0.144416,0.112670,-0.008237,-0.512008,0.856650,-10.545677
...,...,...,...,...,...,...,...
23873,140847239.0,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497
23874,140849891.0,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347
23875,140853397.0,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127
23876,140855323.0,0.233055,-0.229394,0.111900,2.790600,0.246165,-13.841943


## UNIONE DELLE TABELLE
Vengono unite le tabelle relative a IMU, ATT e RCOU, riempiendo i valori mancanti con quelli vicini

In [11]:
m4_fault10 = pd.merge_ordered(imu_m4_fault10,att_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")
m4_fault10 = pd.merge_ordered(m4_fault10, rcou_m4_fault10)
m4_fault10=m4_fault10.fillna(method="ffill").fillna(method="bfill")

Vengono aggiornati i tempi facendo partire il volo all'istante 0, e viene inserita la colonna relativa al guasto, valorizzandola con la variabile *guasto*.

In [12]:
m4_fault10["TimeUS"] = m4_fault10["TimeUS"] - m4_fault10.iloc[0]["TimeUS"]
m4_fault10["TimeUS"] = m4_fault10["TimeUS"].astype(int)
m4_fault10["Guasto"] = guasto

In [ ]:
last_cell = m4_fault10.iloc[-1, m4_fault10.columns.get_loc('TimeUS')]
to_be=[]
for i in range(0, last_cell,2857):
    to_be.append(i)
df = pd.DataFrame(to_be, columns = ['TimeUS'])

Viene corretta la frequenza di campionamento in modo tale da avere 350 misurazioni per ogni secondo

In [ ]:
m4_fault10_final = pd.merge_ordered(m4_fault10,df)
m4_fault10_final=m4_fault10_final.fillna(method="ffill").fillna(method="bfill")
m4_fault10_final = m4_fault10_final[m4_fault10_final.TimeUS.isin(to_be)]

Viene formattata la colonna del tempo in maniera tale che sia più leggibile all'operatore umano

In [19]:
from datetime import timedelta
m4_fault10_final=m4_fault10_final.reset_index(drop=True)
m4_fault10_final['TimeUS'] = pd.to_datetime(m4_fault10_final['TimeUS'], unit='us').dt.strftime('%H:%M:%S.%f')
display(m4_fault10_final)

,TimeUS,GyrX,GyrY,GyrZ,AccX,AccY,AccZ,DesRoll,Roll,DesPitch,Pitch,DesYaw,Yaw,C9,C10,C11,C12,C13,C14,Guasto
0,00:00:00.000000,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,-0.40,355.26,355.355,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
1,00:00:00.002857,-0.562366,-0.017601,0.058196,-0.412550,-0.252978,-9.731380,0.07,-0.46,0.18,-0.40,355.26,355.355,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
2,00:00:00.005714,-0.381477,0.089880,0.034272,-0.460087,0.749301,-10.227750,0.07,-0.46,0.18,-0.40,355.26,355.355,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
3,00:00:00.008571,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,-0.40,355.26,355.355,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
4,00:00:00.011428,0.398408,0.118838,-0.017116,-0.373241,1.252281,-10.283288,0.07,-0.46,0.18,-0.40,355.26,355.355,1454.0,1444.0,1499.0,1395.0,1501.0,1392.0,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24212,00:01:09.173684,0.145880,-0.330572,0.075562,1.462585,0.881795,-12.665910,1.12,0.52,3.70,3.74,1.35,0.390,1581.0,1415.0,1590.0,1403.0,1543.0,1461.0,2.0
24213,00:01:09.176541,0.082320,-0.139260,0.056204,1.207292,0.850404,-12.974497,1.11,0.56,3.62,3.73,1.34,0.450,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0
24214,00:01:09.179398,0.112314,0.057861,0.093426,1.795235,0.927131,-13.250347,1.11,0.56,3.62,3.73,1.34,0.450,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0
24215,00:01:09.182255,0.226368,-0.063633,0.112182,2.494976,0.727650,-13.495127,1.11,0.56,3.62,3.73,1.34,0.450,1558.0,1426.0,1516.0,1474.0,1439.0,1547.0,2.0


## Analisi nel tempo e in frequenza

Funzione che calcola il Root Mean Square

In [20]:
import math
#Funzione che calcola il Root Mean Square (RMS)
def rmsValue(arr, n):
    square = 0
    mean = 0.0
    root = 0.0
    #Calcola il quadrato
    for i in range(0,n):
        square += (arr[i]**2)
    #Calcola la media
    mean = (square / (float)(n))
    #Calcola la radice
    root = math.sqrt(mean)
    return root

Viene definita la funzione generica per ottenere la tabellina con le feauture nel tempo e in frequenza


In [21]:
def time_freq_feat(V):
    N=350
    height_threshold=0.0
    tab= pd.DataFrame([], columns=['tempo',f'<>({V})',f'var({V})',f'kurt({V})',f'RMS({V})',f'freq1({V})',f'freq2({V})',f'amp1({V})',f'amp2({V})'])
    for i in range(int(len(m4_fault10_final)/350)):
        V1=m4_fault10_final[str(V)].iloc[:(i+1*350)+1]
        mean=V1.mean()
        var=V1.var()
        kurt=V1.kurtosis()
        rms=rmsValue(V1,len(V1))
        yf=fft(V1.values)
        xf=fftfreq(N,1/350)
        peaks_index, properties = find_peaks(yf,height=-200,prominence=0.1)
        
        h_max_peak_1=np.unique(properties['peak_heights'])[-1]
        peak1_dimension = np.where(properties['peak_heights'] == h_max_peak_1)
        if (len(peak1_dimension[0])>1):
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][1]
        else:
            index1 = np.where(properties['peak_heights'] == h_max_peak_1)[0][0]

        h_max_peak_2=np.unique(properties['peak_heights'])[-2] 
        peak2_dimension = np.where(properties['peak_heights'] == h_max_peak_2)

        if (len(peak2_dimension[0])>1):
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][1]
        else:
            index2 = np.where(properties['peak_heights'] == h_max_peak_2)[0][0]

        val={'tempo':i,f'<>({V})':mean,f'var({V})':var,f'kurt({V})':kurt,f'RMS({V})':rms,f'freq1({V})':xf[index1],f'freq2({V})':xf[index2],f'amp1({V})':h_max_peak_1,f'amp2({V})':h_max_peak_2}
        tab = tab.astype("float64")
        tab=tab.append(val,ignore_index=True)
    return tab

Si richiama la funzione descritta sopra per ogni variabile in modo da avere le statistiche sia nel tempo che in frequenza

In [22]:
from functools import reduce
GyrX=time_freq_feat('GyrX')
GyrY=time_freq_feat('GyrY')
GyrZ=time_freq_feat('GyrZ')
AccX=time_freq_feat('AccX')	
AccY=time_freq_feat('AccY')
AccZ=time_freq_feat('AccZ')
Roll=time_freq_feat('Roll')
Pitch=time_freq_feat('Pitch')
Yaw=time_freq_feat('Yaw')
c_9 =time_freq_feat('C9')
c_10 =time_freq_feat('C10')
c_11 =time_freq_feat('C11')
c_12 =time_freq_feat('C12')
c_13 =time_freq_feat('C13')
c_14 =time_freq_feat('C14')

merged = [GyrX,GyrY,GyrZ,AccX,AccY,AccZ,Roll,Pitch,Yaw,c_9,c_10,c_11,c_12,c_13,c_14]
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['tempo'],how='outer'), merged)
df_merged["Guasto"] = guasto

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
display(df_merged)

c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_9612\3165731684.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards the imaginary part
  value = np.asarray(value, order='C', dtype=np.float64)
C:\Users\chiar\AppData\Local\Temp\ipykernel_9612\3165731684.py:34: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tab=tab.append(val,ignore_index=True)
c:\Python310\lib\site-packages\scipy\signal\_peak_finding.py:261: ComplexWarning: Casting complex values to real discards th

,tempo,<>(GyrX),var(GyrX),kurt(GyrX),RMS(GyrX),freq1(GyrX),freq2(GyrX),amp1(GyrX),amp2(GyrX),<>(GyrY),var(GyrY),kurt(GyrY),RMS(GyrY),freq1(GyrY),freq2(GyrY),amp1(GyrY),amp2(GyrY),<>(GyrZ),var(GyrZ),kurt(GyrZ),RMS(GyrZ),freq1(GyrZ),freq2(GyrZ),amp1(GyrZ),amp2(GyrZ),<>(AccX),var(AccX),kurt(AccX),RMS(AccX),freq1(AccX),freq2(AccX),amp1(AccX),amp2(AccX),<>(AccY),var(AccY),kurt(AccY),RMS(AccY),freq1(AccY),freq2(AccY),amp1(AccY),amp2(AccY),<>(AccZ),var(AccZ),kurt(AccZ),RMS(AccZ),freq1(AccZ),freq2(AccZ),amp1(AccZ),amp2(AccZ),<>(Roll),var(Roll),kurt(Roll),RMS(Roll),freq1(Roll),freq2(Roll),amp1(Roll),amp2(Roll),<>(Pitch),var(Pitch),kurt(Pitch),RMS(Pitch),freq1(Pitch),freq2(Pitch),amp1(Pitch),amp2(Pitch),<>(Yaw),var(Yaw),kurt(Yaw),RMS(Yaw),freq1(Yaw),freq2(Yaw),amp1(Yaw),amp2(Yaw),<>(C9),var(C9),kurt(C9),RMS(C9),freq1(C9),freq2(C9),amp1(C9),amp2(C9),<>(C10),var(C10),kurt(C10),RMS(C10),freq1(C10),freq2(C10),amp1(C10),amp2(C10),<>(C11),var(C11),kurt(C11),RMS(C11),freq1(C11),freq2(C11),amp1(C11),amp2(C11),<>(C12),var(C12),kurt(C12),RMS(C12),freq1(C12),freq2(C12),amp1(C12),amp2(C12),<>(C13),var(C13),kurt(C13),RMS(C13),freq1(C13),freq2(C13),amp1(C13),amp2(C13),<>(C14),var(C14),kurt(C14),RMS(C14),freq1(C14),freq2(C14),amp1(C14),amp2(C14),Guasto
0,0.0,-0.035348,0.027541,1.454767,0.169445,68.0,67.0,7.807059,4.044368,-0.018257,0.008358,0.200873,0.093101,73.0,77.0,3.839078,2.513268,0.146242,0.006959,-0.956425,0.168301,60.0,66.0,0.792485,0.768133,-0.180379,0.060010,-0.224411,0.303934,113.0,107.0,9.441861,7.813216,0.448292,0.097058,0.716928,0.545662,102.0,106.0,19.016130,12.510048,-10.565958,0.252373,-0.968314,10.577860,0.0,103.0,90.227825,7.175042,0.092023,0.958285,-0.415247,0.981847,85.0,68.0,4.020755,2.213588,-0.216154,0.375083,0.260361,0.648642,85.0,86.0,5.472330,4.745767,213.308960,30361.041415,-1.857867,275.273056,141.0,140.0,9499.683900,3708.598729,1576.481481,4034.993228,2.402646,1577.757066,87.0,89.0,2038.193585,2029.266485,1464.136752,3375.844103,-0.090660,1465.285861,104.0,100.0,3145.432507,1029.450083,1551.803419,1250.969817,0.382598,1552.205288,111.0,109.0,1980.220068,808.101657,1495.803419,2115.221245,-0.401435,1496.508290,105.0,106.0,1340.161060,1057.037653,1499.720798,3288.761823,-0.595737,1500.813734,97.0,96.0,2575.548003,2543.221826,1545.572650,3607.685421,0.694380,1546.735990,100.0,102.0,3068.264005,1333.025360,2
1,1.0,-0.035554,0.027477,1.465293,0.169302,83.0,82.0,8.222915,4.483560,-0.018301,0.008335,0.209094,0.092986,67.0,73.0,3.634382,2.770190,0.145994,0.006960,-0.965500,0.168092,61.0,69.0,0.859417,0.774010,-0.180124,0.059862,-0.218392,0.303540,108.0,105.0,9.763738,7.534831,0.447884,0.096840,0.722739,0.545128,104.0,108.0,17.820520,12.584371,-10.565040,0.251950,-0.968320,10.576923,0.0,105.0,89.973412,7.014630,0.085568,0.970220,-0.418211,0.987312,79.0,76.0,3.990893,2.203599,-0.220227,0.379855,0.223313,0.653663,98.0,99.0,5.687040,4.981714,212.713764,30399.241597,-1.863426,274.881839,142.0,141.0,9442.812445,3867.666067,1576.786932,4056.339088,2.381051,1578.069024,83.0,85.0,2199.426063,2099.498147,1463.852273,3394.713157,-0.095685,1465.008036,99.0,94.0,3301.044649,957.227246,1552.039773,1267.069639,0.355798,1552.446755,106.0,104.0,2020.615289,886.747795,1495.639205,2118.687120,-0.417614,1496.345314,98.0,99.0,1323.190134,1050.606130,1499.977273,3302.546491,-0.600324,1501.074609,96.0,97.0,2715.067967,2667.683193,1545.428977,3604.673004,0.681257,1546.591464,89.0,91.0,3144.777241,1231.604830,2
2,2.0,-0.035574,0.027399,1.478155,0.169077,65.0,83.0,8.921253,3.871360,-0.018556,0.008334,0.200966,0.093032,74.0,75.0,3.753636,2.374775,0.145690,0.006973,-0.977828,0.167866,66.0,74.0,0.898248,0.804823,-0.179866,0.059716,-0.212424,0.303147,107.0,104.0,10.028930,7.188560,0.447867,0.096565,0.733286,0.544863,92.0,96.0,16.313584,12.767695,-10.563528,0.252040,-0.974395,10.575418,0.0,108.0,89.922057,7.058613,0.078669,0.984268,-0.409383,0.993815,89.0,86.0,3.777864,2.030420,-0.223938,0.383635,0.180723,0.657797,83.0,84.0,5.998496,5.329771,212.121997,30436.4

## Output
Salviamo la tabella ottenuta in un file csv

In [23]:
path_file = path_file.replace(r"M4\FAULT_M4_10", "")
os.chdir(path_file)
df_merged.to_csv('m4_fault10.csv', index=False)
print(path_file)

c:\Users\chiar\OneDrive\Desktop\MantenzionePreventiva\ProgettoManutenzionePreventiva\
